In [6]:
import numpy as np

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import tempfile

from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torchvision.models import vgg19_bn

from tqdm.notebook import tqdm_notebook as tqdm

# Data

In [2]:
cifar10 = CIFAR10(tempfile.gettempdir(),
                  train=True,
                  download=True)
_mean = cifar10.data.mean(axis=0).mean(0).mean(0)/255
_std = cifar10.data.std(axis=0).std(0).std(0)/255

print('normalization mean:', _mean)
print('normalization std :', _std)


transform = transforms.Compose([transforms.RandomCrop(32, padding=3),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomVerticalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize(_mean, _std)])

train_loader = DataLoader(CIFAR10(tempfile.gettempdir(),
                                  train=True,
                                  transform=transform,
                                  download=True),
                          batch_size=64,
                          shuffle=True)

test_loader = DataLoader(CIFAR10(tempfile.gettempdir(),
                                 train=False,
                                 transform=transform,
                                 download=True),
                         num_workers=4,
                         batch_size=64,
                         shuffle=True,
                         drop_last=True, 
                         pin_memory=True)

Extracting /tmp/cifar-10-python.tar.gz to /tmp
normalization mean: [0.49139968 0.48215841 0.44653091]
normalization std : [0.00089181 0.00114635 0.00057204]
Files already downloaded and verified
Files already downloaded and verified


# Model

In [5]:
class Model(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        backbone = vgg19_bn(pretrained=True)
        backbone = nn.Sequential(* (list(list(backbone.modules())[0].children())[:-2]))
        
#         for param  in list(backbone.parameters())[:-25]:
#             param.requires_grad = False
                    
        self.backbone = backbone
        self.gap = nn.AdaptiveAvgPool2d((1,1))
        
        self.linear = nn.Linear(512, 10)
        
    def forward(self, x):
        h1 = self.backbone(x)
        h2 = self.gap(h1)
        b_sz, c_sz, h_sz, w_sz = h2.shape
        h2 = h2.view(b_sz, c_sz)
        h3 = self.linear(h2)
        output = F.log_softmax(h3)
        return output

clf = Model()
clf = clf.cuda()
adam = Adam(clf.parameters(),  lr=0.01)
lr_scheduler = ReduceLROnPlateau(adam, factor=0.1)

# Train

In [7]:
def train_epoch(clf, opt, train_loader, epoch_size:int=500):
    clf.train()
    n_samples = 0
    loss_data = []
    n_corrects = []
    
    prev_param = None
    for batch_id, (data, label) in tqdm(enumerate(train_loader), total=len(train_loader)):
        data = data.cuda()
        label = label.cuda()
        
#         if batch_id %100 == 0:
#             param = torch.Tensor([round(p.mean().item(), 2) for p in clf.parameters()])
#             if prev_param is not None:
#                 diff = torch.where(prev_param != param)
#                 if len(param[diff]):
#                     print(diff)
#             prev_param = param
            
        # Forward
        opt.zero_grad()
        preds = clf(data)
        pred_probs, pred_idx = preds.max(1)

        # Loss
        loss = F.nll_loss(preds, label) # Negative Log Loss
        loss.backward()
        adam.step()
        
        # Log
        n_samples += len(label)
        n_corrects.append((pred_idx == label).sum().item())
        loss_data.append(loss.item())
        
    
    return {'loss': np.mean(loss_data), 
            'acc': np.sum(n_corrects)/n_samples}

def test_epoch(clf, test_loader):
    clf.eval() # set model in inference mode (need this because of dropout)
    n_samples = 0
    n_corrects = []
    test_losses = []
    
    
    for data, target in test_loader:
        data = data.cuda()
        target = target.cuda()
        
        output = clf(data)
        test_loss = F.nll_loss(output, target)
        pred_probs, pred_idx = output.max(1)
        
        # Log
        n_samples += len(target)
        n_corrects.append((pred_idx == target).sum().item())
        test_losses.append(test_loss.item())
    
    return {'loss': np.mean(test_losses), 
            'acc': np.sum(n_corrects)/n_samples}

        
def train(epoch_size=1000):
    if not os.path.exists('./checkpoints'):
        os.makedirs('./checkpoints')
    
    best_val_acc = 0
    for epoch in range(epoch_size):
        res_train = train_epoch(clf, adam, train_loader)  
        res_test = test_epoch(clf, test_loader)
        
        lr_scheduler.step(res_test['loss'])
        last_lr = lr_scheduler.optimizer.param_groups[0]['lr']
        print(f'epoch:{epoch:02} | loss:{res_train["loss"]:.2f}, {res_test["loss"]:.2f} | '
              f'acc:{res_train["acc"]:.2f}, {res_test["acc"]:.2f} | lr:{last_lr:6.4f}')
        
        # Save
        if res_test['acc'] > best_val_acc:
            best_val_acc = res_test['acc']
            save_path = os.path.join(f'./checkpoints/checkpoint_{epoch}_{best_val_acc:.2f}.pth')
            torch.save(model.state_dict(), save_path)
        
train(100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



epoch:00 | loss:2.10, 1.87 | acc:0.19, 0.27 | lr:0.0100


NameError: name 'model' is not defined